In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [3]:
# Summarization
# Stuff: 전체 문서 한 번에 요약.
#        단순히 모든 문서를 단일 프롬프트로 "넣는" 방법. 이는 가장 간단한 접근 방식입니다.

# stuff documents chain은 문서 체인 중 가장 간단한 방식입니다. 
# 문서 목록을 가져와서 모두 프롬프트에 삽입한 다음, 그 프롬프트를 LLM에 전달합니다.
# 이 체인은 문서가 작고 대부분의 호출에 몇 개만 전달되는 애플리케이션에 적합합니다.

In [4]:
from langchain_community.document_loaders import TextLoader

In [7]:
# 뉴스데이터 로드
loader = TextLoader("./data/news.txt", encoding='utf-8')
docs = loader.load()

In [8]:
print(f"총 글자수: {len(docs[0].page_content)}")

총 글자수: 1708


In [9]:
print("\n========= 앞부분 미리보기 =========\n")
print(docs[0].page_content[:500])


========= 앞부분 미리보기 =========

제목: 
AI2, 상업 활용까지 자유로운 '진짜' 오픈 소스 LLM '올모' 출시

내용:
앨런AI연구소(AI2)가 완전한 오픈 소스 대형언어모델(LLM) '올모(OLMo)’를 출시했다. 데이터 수집, 학습, 배포의 전 과정을 투명하게 공개한 데다 상업적 사용까지 허용한 진정한 의미의 오픈 소스 LLM이라는 평가다.
벤처비트는 1일(현지시간) 비영리 민간 AI 연구기관인 AI2가 ‘최초의 진정한 오픈 소스 LLM 및 프레임워크’라고 소개한 ‘올모’를 출시했다고 보도했다. 
이에 따르면 올모는 모델 코드와 모델 가중치뿐만 아니라 훈련 코드, 훈련 데이터, 관련 툴킷 및 평가 툴킷도 제공한다. 이를 통해 모델이 어떻게 구축되었는지 심층적으로 분석, LLM의 작동 방식과 응답을 생성하는 원리를 더 잘 이해할 수 있다. 
올모 프레임워크는 70억 매개변수의 ‘올모 7B’ 등 4가지 변형 모델과 10억 매개변수의 ‘올모 1B’ 모델을 제공한다. 모델들은 훈련 데이터를 생성하는 코드를 포함해 


In [11]:
from langchain import hub

# 한국어로 요약을 작성하라는 문구가 추가된 prompt 입니다.
prompt = hub.pull("teddynote/summary-stuff-documents-korean")
prompt.pretty_print()

Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. Translate the summary into KOREAN if it is written in ENGLISH.
5. DO NOT translate any technical terms.
6. DO NOT include any unnecessary information.

CONTEXT:
{context}

SUMMARY:"



In [12]:
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_teddynote.callbacks import StreamingCallback

In [13]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1, 
    streaming=True,
    callbacks=[StreamingCallback()]
)

In [14]:
stuff_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
answer = stuff_chain.invoke({"context": docs})

- 🚀 앨런AI연구소(AI2)가 완전한 오픈 소스 LLM '올모(OLMo)'를 출시했다.  
- 📊 데이터 수집, 학습, 배포 과정을 투명하게 공개하고 상업적 사용을 허용한다.  
- 🔍 올모는 모델 코드, 가중치, 훈련 코드, 데이터 및 평가 툴킷을 제공한다.  
- 🧠 70억 매개변수의 '올모 7B'와 10억 매개변수의 '올모 1B' 모델을 포함한 4가지 변형 모델이 있다.  
- 📈 훈련 데이터는 3조개의 토큰으로 구성된 AI2의 '돌마(Dolma)' 데이터 세트를 기반으로 한다.  
- 📝 아파치 2.0 라이선스에 따라 상업적 활용에 제한이 없다.  
- 💬 AI2의 프로젝트 책임자는 투명성이 부족한 현재의 언어 모델에 대한 문제를 지적했다.  
- 🌟 올모는 상업용 제품과 동등한 성능을 보여주며, 일부 벤치마크에서 우수한 성과를 기록했다.  
- ⚠️ 비영어권 언어와 코드 생성 기능에 대한 제약이 있다.  
- 🔧 AI2는 올모를 지속적으로 개선할 계획이다.  
- 🌐 모든 리소스는 깃허브 및 허깅페이스에서 무료로 제공된다.  

In [16]:
llm2 = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0,
    callbacks=[StreamingCallback()]
)

In [17]:
stuff_chain2 = create_stuff_documents_chain(llm2, prompt)

In [18]:
answer2 = stuff_chain2.invoke({"context": docs})

In [19]:
print(answer2)

- 🚀 앨런AI연구소(AI2)가 완전한 오픈 소스 LLM '올모(OLMo)'를 출시했다.  
- 🔍 데이터 수집, 학습, 배포 과정을 투명하게 공개하고 상업적 사용을 허용한다.  
- 📊 올모는 모델 코드, 가중치, 훈련 코드, 데이터 및 평가 툴킷을 제공한다.  
- 🧩 70억 매개변수의 '올모 7B'와 10억 매개변수의 '올모 1B' 모델을 포함한 4가지 변형 모델이 있다.  
- 📈 AI2의 '돌마(Dolma)' 데이터 세트를 기반으로 3조개의 토큰으로 사전 훈련되었다.  
- 📜 아파치 2.0 라이선스에 따라 상업적 활용에 제한이 없다.  
- 🧪 올모는 상업용 제품과 동등한 성능을 보여주며, 일부 벤치마크 테스트에서 우수한 성과를 기록했다.  
- 🌍 비영어권 언어에 대한 낮은 품질과 약한 코드 생성 기능이 제약 사항으로 지적되었다.  
- 🔧 AI2는 올모를 계속해서 향상할 계획이다.  
- 🌐 현재 올모의 모든 리소스는 깃허브 및 허깅페이스에서 무료로 제공된다.  
